In [1]:
from sklearn.svm import SVC
from ml_pipeline import MLPipeline

# SVM with Wavelet Transform (subset_ratio=1/60)

In [2]:
# Initialiser le pipeline avec SVC
pipeline = MLPipeline(model=SVC(), wavelet='bior1.3')

# Charger les données avec subset_ratio=1/60
pipeline.load_data(
    train_path='./data/fashion-mnist_train.csv',
    test_path='./data/fashion-mnist_test.csv',
    subset_ratio=1/1
)

╭─────────── 📊 Loading Data ───────────╮
│ Loading datasets...                   │
│ Train: ./data/fashion-mnist_train.csv │
│ Test: ./data/fashion-mnist_test.csv   │
╰───────────────────────────────────────╯

⚠️  Applying subset ratio: 100.00%

         Dataset Summary          
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Metric               ┃ Value   ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ Train samples        │ 60000   │
│ Test samples         │ 10000   │
│ Features per sample  │ 784     │
│ Subset ratio applied │ 100.00% │
└──────────────────────┴─────────┘

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✅ Data loaded successfully!                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

MLPipeline(model=SVC, wavelet='bior1.3', trained=False)

In [ ]:
# GridSearchCV avec SVM
param_grid = {
    'C': [0.1, 1],
    'kernel': ['poly', 'linear']
}

pipeline.fit_with_gridsearch(
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    use_wavelets=True,
    normalize=True,
    verbose=3,
    use_scaler=True,
)

╭── 🔍 GridSearchCV Hyperparameter Tuning ──╮
│ Model: SVC                                │
│ Cross-validation folds: 5                 │
│ Scoring metric: accuracy                  │
│ Parameter combinations: 4                 │
│ Total fits: 20 (4 combinations × 5 folds) │
│ Use wavelets: Yes                         │
│ Normalize: Yes                            │
╰───────────────────────────────────────────╯

               Parameter Grid                
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Parameter ┃ Values             ┃ # Values ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ C         │ [0.1, 1]           │    2     │
│ kernel    │ ['poly', 'linear'] │    2     │
└───────────┴────────────────────┴──────────┘

Preparing features...

Output()

Starting GridSearchCV with 5-fold cross-validation...

...

Tip: Watch the output below for sklearn's verbose progress

═══════════════════════════════════════════════════════════

             GridSearchCV Progress Output                  

═══════════════════════════════════════════════════════════

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [ ]:
# Évaluation sur le jeu de test
pipeline.predict()
pipeline.evaluate()